<a href="https://colab.research.google.com/github/Mehraj71/PhonePe-Pulse-Data-Visualization-/blob/main/PhonePe_Pulse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PhonePe Pulse Data Visualization and Exploration

**Step 1: Extracting data from PhonePe Pulse github repository**



In [1]:
#cloning the repository from github
! git clone "https://github.com/PhonePe/pulse"

Cloning into 'pulse'...
remote: Enumerating objects: 17904, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 17904 (delta 19), reused 17 (delta 17), pack-reused 17855 (from 2)
Receiving objects: 100% (17904/17904), 26.13 MiB | 18.57 MiB/s, done.
Resolving deltas: 100% (8723/8723), done.
Updating files: 100% (9029/9029), done.


**Step 2: Transforming Data into suitable format and performing any necessary cleaning and pre-processing steps.**

In [2]:
import pandas as pd
import json
import os
path1 = "/content/pulse/data/aggregated/transaction/country/india/state"
#print(path1)
agg_state_list = os.listdir(path1)
#print(agg_state_list)
clm = {"state":[],"year":[],"quater":[],"Transaction_Type":[],"Transaction_count":[],"Transaction_amount":[]}
for i in agg_state_list:
  p_i = path1+"/"+i
  agg_yr = os.listdir(p_i)
  #print(agg_yr)
  for j in agg_yr:
    p_j = p_i+"/"+j
    agg_yr_quater = os.listdir(p_j)
    #print(agg_yr)
    for k in agg_yr_quater:
      p_k = p_j+"/"+k
      data = open(p_k,'r')
      #print(data)
      D = json.load(data)
      for z in D['data']['transactionData']:
        Name = z['name']
        count = z['paymentInstruments'][0]['count']
        amount = z['paymentInstruments'][0]['amount']
        clm['Transaction_Type'].append(Name)
        clm['Transaction_count'].append(count)
        clm['Transaction_amount'].append(amount)
        clm['state'].append(i)
        clm['year'].append(j)
        clm['quater'].append(int(k.strip('.json')))


**Creating a Dataframe**

In [3]:
df_agg_transaction = pd.DataFrame(clm)
df_agg_transaction.head()

,state,year,quater,Transaction_Type,Transaction_count,Transaction_amount
0,karnataka,2021,3,Merchant payments,391381353,1.905634e+11
1,karnataka,2021,3,Peer-to-peer payments,288462867,9.081386e+11
2,karnataka,2021,3,Recharge & bill payments,72168311,3.338671e+10
3,karnataka,2021,3,Financial Services,286685,5.080518e+08
4,karnataka,2021,3,Others,997323,6.379270e+08


In [4]:
pd.DataFrame(clm)
dfAg = pd.DataFrame(clm)
dfAg
#Successfully created a DataFrame

,state,year,quater,Transaction_Type,Transaction_count,Transaction_amount
0,karnataka,2021,3,Merchant payments,391381353,1.905634e+11
1,karnataka,2021,3,Peer-to-peer payments,288462867,9.081386e+11
2,karnataka,2021,3,Recharge & bill payments,72168311,3.338671e+10
3,karnataka,2021,3,Financial Services,286685,5.080518e+08
4,karnataka,2021,3,Others,997323,6.379270e+08
...,...,...,...,...,...,...
5029,himachal-pradesh,2024,1,Merchant payments,23888813,1.295578e+10
5030,himachal-pradesh,2024,1,Peer-to-peer payments,11704961,3.264583e+10
5031,himachal-pradesh,2024,1,Recharge & bill payments,3859407,2.212162e+09
5032,himachal-pradesh,2024,1,Financial Services,30691,4.898123e+07


**Inserting the DataFrame into sqlite format**

In [5]:
import sqlite3
con = sqlite3.connect('pulse.db')
c = con.cursor()

In [6]:
#Create a table in database
c.execute('''CREATE TABLE IF NOT EXISTS transaction_data(
            State TEXT,
            Year TEXT,
            Quater TEXT,
            Transacion_type TEXT,
            Transacion_count TEXT,
            Transacion_amount TEXT
            )''')

In [7]:
#insert the All data into the table
c.executemany('INSERT INTO transaction_data VALUES(?,?,?,?,?,?)',
               zip(clm['state'], clm['year'],clm['quater'],clm['Transaction_Type'], clm['Transaction_count'], clm['Transaction_amount']))

In [8]:
#Show the data into table format
x=c.execute("SELECT * FROM transaction_data")
x=c.fetchall()
for row in x:
  print(row)

('karnataka', '2021', '3', 'Merchant payments', '391381353', '190563423840.345')
('karnataka', '2021', '3', 'Peer-to-peer payments', '288462867', '908138627418.701')
('karnataka', '2021', '3', 'Recharge & bill payments', '72168311', '33386709060.8071')
('karnataka', '2021', '3', 'Financial Services', '286685', '508051800.862504')
('karnataka', '2021', '3', 'Others', '997323', '637927036.873722')
('karnataka', '2021', '4', 'Merchant payments', '503210623', '243754536471.111')
('karnataka', '2021', '4', 'Peer-to-peer payments', '350032663', '1090951587881.04')
('karnataka', '2021', '4', 'Recharge & bill payments', '76826701', '39294350717.4059')
('karnataka', '2021', '4', 'Financial Services', '365769', '574240588.188319')
('karnataka', '2021', '4', 'Others', '1289682', '896608692.124491')
('karnataka', '2021', '2', 'Merchant payments', '244460862', '124624417935.059')
('karnataka', '2021', '2', 'Peer-to-peer payments', '216539959', '684035748184.438')
('karnataka', '2021', '2', 'Recharg

In [9]:
c.executemany('INSERT INTO transaction_data VALUES(?,?,?,?,?,?)',
               zip(clm['state'], clm['year'],clm['quater'],clm['Transaction_Type'], clm['Transaction_count'], clm['Transaction_amount']))

In [10]:
#fetch the data from the table where the state is Tamil Nadu and year is 2018 and quater is 1 and transaction type is Recharge and bill payments
d=c.execute("SELECT * FROM transaction_data WHERE state = 'Tamil Nadu' AND year =  2018 AND quater = 1 AND 'transaction type' = 'Recharge & bill payments' AND 'transaction_amount'>0")
d=c.fetchall()
for row in d:
  print(row)

In [11]:
#commit changes and close connection
con.commit()
con.close()

**Extracting the data and creating a DataFrame path2**

In [12]:
path2 = "/content/pulse/data/aggregated/user/country/india/state"
#print(path2)
Agg_transc_user_list = os.listdir(path2)
#print(Agg_transc_user_list)
clm = clm={"No of Registered User":[],"AppOpens":[],'State':[], 'Year':[], "Quarter":[]}

for i in Agg_transc_user_list:
  p_i = path2+"/"+i
  Agg_year = os.listdir(p_i)
  #print(Agg_year)
  Agg_year = os.listdir(p_i)
  for j in Agg_year:
    p_j = p_i +"/"+j
    Agg_year_list = os.listdir(p_j)
    #print(Agg_year_list)
    for k in Agg_year_list:
      p_k = p_j +"/"+k
      with open(p_k,'r') as f:
        data = json.load(f)
        clm['No of Registered User'].append(data['data']['aggregated']['registeredUsers'])
        clm['AppOpens'].append(data['data']['aggregated']['appOpens'])
        #clm['Percentage'].append(data['data']['usersByDevice'][0]['percentage'])
        #clm['Brand'].append(data['data']['usersByDevice'])
        clm['State'].append(i)
        clm['Year'].append(j)
        clm['Quarter'].append(int(k.strip('.json')))

pd_agg_user = pd.DataFrame(clm)

In [13]:
pd_agg_user.head()

,No of Registered User,AppOpens,State,Year,Quarter
0,27614221,1366781087,karnataka,2021,3
1,29459646,1941558513,karnataka,2021,4
2,25751023,1051089714,karnataka,2021,2
3,24095832,1084917385,karnataka,2021,1
4,34493994,2148016434,karnataka,2022,3


In [14]:
con = sqlite3.connect('transaction_data.db')
a = con.cursor()

**Creating a Table in Database**

In [15]:
a.execute('''CREATE TABLE IF NOT EXISTS transaction_data(
             No_of_Registered_User TEXT,
             AppOpens TEXT,
             State TEXT,
             Year TEXT,
             Quarter TEXT
             )''')

In [16]:
a.executemany('INSERT INTO transaction_data VALUES(?,?,?,?,?)',
               zip(clm['No of Registered User'],clm['AppOpens'],clm['State'],clm['Year'],clm['Quarter']))

In [17]:
a.execute("SELECT * FROM transaction_data")

#Fetch all the rows in the result set
rows = a.fetchall()

#Loop through each row and print the data
for row in rows:
    print(row)

# Close the connection
#a.close()
# con.close()

('27614221', '1366781087', 'karnataka', '2021', '3')
('29459646', '1941558513', 'karnataka', '2021', '4')
('25751023', '1051089714', 'karnataka', '2021', '2')
('24095832', '1084917385', 'karnataka', '2021', '1')
('34493994', '2148016434', 'karnataka', '2022', '3')
('36081698', '3011645011', 'karnataka', '2022', '4')
('32831447', '2065359837', 'karnataka', '2022', '2')
('31200349', '1727054045', 'karnataka', '2022', '1')
('20487550', '693683212', 'karnataka', '2020', '3')
('22400182', '935510690', 'karnataka', '2020', '4')
('18655366', '490510075', 'karnataka', '2020', '2')
('17016980', '626488036', 'karnataka', '2020', '1')
('13635075', '445683660', 'karnataka', '2019', '3')
('15385416', '569674870', 'karnataka', '2019', '4')
('11968087', '113024811', 'karnataka', '2019', '2')
('10266174', '0', 'karnataka', '2019', '1')
('6372649', '0', 'karnataka', '2018', '3')
('8315802', '0', 'karnataka', '2018', '4')
('5030665', '0', 'karnataka', '2018', '2')
('3717763', '0', 'karnataka', '2018', '

**Path3- Extracting json data into dataframe -- map-->transaction-->hover-->country-->india-->state**


In [18]:
path3 = "/content/pulse/data/map/transaction/hover/country/india/state/"
map_state_list = os.listdir(path3)

# This is to extract the data's to create a dataframe
clm = {'State': [], 'district':[],'Year': [], 'Quarter': [], 'Metric_type': [], 'Metric_count': [], 'Metric_amount': []}
for i in map_state_list:
    p_i = path3 + i + "/"
    Agg_yrs = os.listdir(p_i)
    for j in Agg_yrs:
        p_j = p_i + j + "/"
        Agg_yrs_list = os.listdir(p_j)
        for k in Agg_yrs_list:
            p_k = p_j + k
            with open(p_k, 'r') as f:
                data = json.load(f)
                clm['district'].append(data['data']['hoverDataList'][0]['name'])
                clm['State'].append(i)
                clm['Quarter'].append(j)
                clm['Year'].append(int(k.strip('.json')))
                for metric in data['data']['hoverDataList'][0]['metric']:
                    clm['Metric_type'].append(metric['type'])
                    clm['Metric_count'].append(metric['count'])
                    clm['Metric_amount'].append(metric['amount'])

df3 = pd.DataFrame(clm)

In [19]:
#Checking for the null values
df3.isnull().sum()

,0
State,0
district,0
Year,0
Quarter,0
Metric_type,0
Metric_count,0
Metric_amount,0


In [20]:
conn = sqlite3.connect("Map_Transaction_india.db")
s = conn.cursor()

**Creating a table in Database**

In [21]:
s.execute('''CREATE TABLE IF NOT EXISTS Map_Transaction_india(
            State TEXT,
            district TEXT,
            Year TEXT,
            Quarter TEXT,
            Metric_type TEXT,
            Metric_count TEXT,
            Metric_amount TEXT
            )''')

In [22]:
s.executemany('INSERT INTO Map_Transaction_india VALUES(?,?,?,?,?,?,?)',
             zip(clm['State'],clm['district'],clm['Year'],clm['Quarter'],clm['Metric_type'],clm['Metric_count'],clm['Metric_amount']))

In [23]:
conn = sqlite3.connect('Map_Transaction_india.db')
c = conn.cursor()

In [24]:
c.execute("SELECT * FROM Map_Transaction_india")

# Fetch all the rows in the result set
rows = s.fetchall()

# Loop through each row and print the data
for row in rows:
    print(row)

conn.commit()
# Close the connection
conn.close()

**Path4-- #This is to direct the path to get the data as states map-->user-->hover-->country-->india-->state**

In [25]:
path4 = "/content/pulse/data/map/user/hover/country/india/state/"
map_state_list = os.listdir(path4)

clm = {'State': [], 'Year': [], 'Quarter': [], 'No of users': [], 'AppOpen': [], 'Area': []}

for i in map_state_list:
    p_i = os.path.join(path4, i)
    Agg_yrs = os.listdir(p_i)

    for j in Agg_yrs:
        p_j = os.path.join(p_i, j)
        Agg_yrs_list = os.listdir(p_j)

        for k in Agg_yrs_list:
            p_k = os.path.join(p_j, k)
            with open(p_k, 'r') as f:
                data = json.load(f)

                # Safely extract hover data
                hover_data = data.get('data', {}).get('hoverData', {})

                total_users = 0
                total_app_opens = 0

                for district_info in hover_data.values():
                    total_users += district_info.get('registeredUsers', 0)
                    total_app_opens += district_info.get('appOpens', 0)

                clm['State'].append(i)
                clm['Year'].append(j)
                clm['Quarter'].append(int(k.strip('.json')))
                clm['No of users'].append(total_users)
                clm['AppOpen'].append(total_app_opens)
                clm['Area'].append(None)  # Still optional, since it's unclear what Area means

df4 = pd.DataFrame(clm)


In [26]:
con = sqlite3.connect("map_state_list.db")
s = con.cursor()

In [27]:
df4.isnull().sum()

,0
State,0
Year,0
Quarter,0
No of users,0
AppOpen,0
Area,1008


**Creating a table in Database**

In [28]:
s.execute('''CREATE TABLE IF NOT EXISTS map_state_list(
            State TEXT,
            Year TEXT,
            Quarter TEXT,
            No of users TEXT,
            AppOpen TEXT,
            Area TEXT
            )''')

In [29]:
s.executemany('INSERT INTO map_state_list VALUES(?,?,?,?,?,?)',
              zip(clm['State'],clm['Year'],clm['Quarter'],clm['No of users'],clm['AppOpen'],clm['Area']))

In [30]:
# Loop through each row and print the data
for row in rows:
    print(row)


#conn.commit()
# Close the connection
conn.close()


**Path5**

In [31]:
path5 = "/content/pulse/data/top/transaction/country/india/state/"
map_state_lists = os.listdir(path5)

clm = {'State': [], 'district': [], 'Year': [], 'Quarter': [], 'Metric_type': [], 'Metric_count': [], 'Metric_amount': []}

for state in map_state_lists:
    state_path = os.path.join(path5, state)
    if not os.path.isdir(state_path):
        continue
    years = os.listdir(state_path)

    for year in years:
        year_path = os.path.join(state_path, year)
        if not os.path.isdir(year_path):
            continue
        quarters = os.listdir(year_path)

        for quarter_file in quarters:
            file_path = os.path.join(year_path, quarter_file)
            if not quarter_file.endswith('.json'):
                continue
            try:
                with open(file_path, 'r') as f:
                    data = json.load(f)

                    districts = data['data'].get("districts", [])
                    if districts:
                        district_data = districts[0]
                        clm['State'].append(state)
                        clm['district'].append(district_data.get('entityName'))
                        clm['Year'].append(year)
                        clm['Quarter'].append(quarter_file.strip('.json'))
                        metric = district_data.get('metric', {})
                        clm['Metric_type'].append(metric.get('type', 'N/A'))
                        clm['Metric_count'].append(metric.get('count', 0))
                        clm['Metric_amount'].append(metric.get('amount', 0))
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

In [32]:
# Converting into DataFrame
df5 = pd.DataFrame(clm)

In [33]:
# Creating SQLite connection
con = sqlite3.connect("map_state_lists.db")
s = con.cursor()

In [34]:
# Creating a table
s.execute('''
    CREATE TABLE IF NOT EXISTS map_state_lists(
        State TEXT,
        district TEXT,
        Year TEXT,
        Quarter TEXT,
        Metric_type TEXT,
        Metric_count TEXT,
        Metric_amount TEXT
    )
''')

In [35]:
# Insert data into table
s.executemany('INSERT INTO map_state_lists VALUES (?, ?, ?, ?, ?, ?, ?)',
              zip(clm['State'], clm['district'], clm['Year'], clm['Quarter'], clm['Metric_type'], clm['Metric_count'], clm['Metric_amount']))

In [36]:
con = sqlite3.connect("map_state_lists.db")
s = con.cursor()

In [37]:
# Commit and close
con.commit()
con.close()

In [38]:
path6 = "/content/pulse/data/top/user/country/india/state/"
map_states_list = os.listdir(path6)

# This is to extract the data's to create a dataframe
clm = {'State': [], 'district':[],'Year': [], 'Quarter': [], 'No of Regi_user':[]}
for i in map_states_list:
    p_i = path6 + i + "/"
    Agg_yrs = os.listdir(p_i)
    for j in Agg_yrs:
        p_j = p_i + j + "/"
        Agg_yrs_list = os.listdir(p_j)
        for k in Agg_yrs_list:
            p_k = p_j + k
            with open(p_k, 'r') as f:
                data = json.load(f)
                clm['State'].append(i)
                clm['Quarter'].append(j)
                clm['Year'].append(int(k.strip('.json')))
                clm['district'].append(data['data']['districts'][0]['name'])
                clm['No of Regi_user'].append(data['data']['districts'][0]['registeredUsers'])
df6 = pd.DataFrame(clm)

In [39]:
# convering into dataframe
df5 = pd.DataFrame(clm)

In [40]:
# Creating SQLite connection
con = sqlite3.connect("map_states_lists.db")
s = con.cursor()

In [41]:
# Creating a table
s.execute('''
    CREATE TABLE IF NOT EXISTS map_states_lists(
        State TEXT,
        Quarter TEXT,
        Year TEXT,
        district TEXT,
        No of Regi_user TEXT

    )
''')

In [42]:
con = sqlite3.connect("map_states_lists.db")
s = con.cursor()

In [43]:
# Commit and close
con.commit()
con.close()

In [ ]:
!pip install streamlit